# 4v7g

In [1]:
using Printf, DelimitedFiles
using Plots
using DataFrames
using Revise, MDToolbox

In [2]:
function read_ascii(fname)
    lines = readlines(fname)
    natom = length(lines) - 1
    xyz = zeros(Float64, 1, natom*3)
    for i = 1:natom
        xyz[1, 3*(i-1)+1] = parse(Float64, lines[i+1][31:38])
        xyz[1, 3*(i-1)+2] = parse(Float64, lines[i+1][39:46])
        xyz[1, 3*(i-1)+3] = parse(Float64, lines[i+1][47:54])
    end
    TrjArray(xyz=xyz)
end

read_ascii (generic function with 1 method)

In [11]:
fnames = []
omega = []
phi = []
prou = []
d = []
score = []
rmsd = []
struc = []

Any[]

In [12]:
nrun = 1000

1000

In [13]:
for irun = 1:nrun

    if irun % 100 == 0
        print("irun = $irun")
    end
    for i in 1:100
        filename = "$(irun)/complex.$(i).pdb"
        #t = mdload(filename)
        #t = MDToolbox.readpdb_biostructures(filename)
        t = read_ascii(filename)
        t1 = t[:, 1:1129]
        t2 = t[:, 1130:end]
        r = MDToolbox.compute_skrew(t1, t2)
        push!(fnames, filename)
        push!(omega, r.omega)
        push!(phi, r.phi / (2.0 * pi) * 360.0)
        push!(prou, r.prou)
        push!(d, r.d)
        push!(struc, t)
        #t_ca = t["atomname CA"]
        #push!(rmsd, compute_rmsd_permute(ref, t_ca, 2))
    end

    s = readdlm("$(irun)/zdock.out", skipstart=5)
    score = [score; s[1:100, end]]
end

irun = 100irun = 200irun = 300irun = 400irun = 500irun = 600irun = 700irun = 800irun = 900irun = 1000

In [14]:
df = DataFrame(fnames=fnames, score=score, omega=omega, phi=phi, prou=prou, d=d, struc=struc)

Row,fnames,score,omega,phi,prou,d,struc
,Any,Any,Any,Any,Any,Any,Any
1,1/complex.1.pdb,1476.6,"[-0.173076, -0.791418, -0.586261]",72.632,"[53.6754, 11.9727, -32.0085]",0.846014,"1x2258 TrjArray{Float64, Int64}\n| 93.66 54.74 -5.40 | … 74.34 37.26 3.22 |"
2,1/complex.2.pdb,1434.06,"[0.546049, -0.425146, 0.721859]",154.985,"[68.9845, 72.8023, -9.30562]",-22.2624,"1x2258 TrjArray{Float64, Int64}\n| 93.66 54.74 -5.40 | … 67.70 97.60 -9.68 |"
3,1/complex.3.pdb,1433.56,"[0.0914852, 0.842105, 0.531497]",67.053,"[58.1435, 10.2176, -26.1969]",2.98047,"1x2258 TrjArray{Float64, Int64}\n| 93.66 54.74 -5.40 | … 58.00 58.68 -18.17 |"
4,1/complex.4.pdb,1358.04,"[0.415051, 0.536264, -0.734951]",169.218,"[32.2834, 45.9905, 51.7889]",-6.48095,"1x2258 TrjArray{Float64, Int64}\n| 93.66 54.74 -5.40 | … 36.32 111.19 15.44 |"
5,1/complex.5.pdb,1341.26,"[-0.348243, -0.511949, 0.785261]",161.249,"[39.1631, 51.1556, 50.7187]",-6.61469,"1x2258 TrjArray{Float64, Int64}\n| 93.66 54.74 -5.40 | … 59.19 121.57 15.42 |"
6,1/complex.6.pdb,1322.27,"[-0.62595, -0.691659, -0.36027]",174.545,"[26.2185, -3.30103, -39.2159]",-0.118216,"1x2258 TrjArray{Float64, Int64}\n| 93.66 54.74 -5.40 | … 73.19 51.31 -7.41 |"
7,1/complex.7.pdb,1309.68,"[0.230504, -0.969612, -0.0819839]",166.598,"[81.6898, 20.3154, -10.5903]",31.5404,"1x2258 TrjArray{Float64, Int64}\n| 93.66 54.74 -5.40 | … 80.54 17.75 -4.07 |"
8,1/complex.8.pdb,1304.88,"[0.526833, 0.760823, 0.378939]",175.718,"[37.3555, -6.36759, -39.1501]",1.80582,"1x2258 TrjArray{Float64, Int64}\n| 93.66 54.74 -5.40 | … 75.64 50.95 -7.34 |"
9,1/complex.9.pdb,1285.13,"[0.128234, 0.306095, -0.943325]",178.053,"[55.6515, 70.5797, 30.4672]",-0.0974977,"1x2258 TrjArray{Float64, Int64}\n| 93.66 54.74 -5.40 | … 42.34 113.89 1.62 |"


In [15]:
df_org = deepcopy(df);

In [16]:
#using BSON: @save, @load
#@save "skrew_parameters.bson" fnames score omega phi prou d df rmsd

In [17]:
#using BSON: @save, @load
#@save "df.bson" df

In [18]:
using JLD2

save("skrew_parameters.jld2", "fnames", fnames, "score", score, "omega", omega, "phi", phi, "prou", prou, "d", d, "df", df, "rmsd", rmsd)
save("df.jld2", "df", df)